In [1]:
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
from pyscf.scf import cphf 
# ancora il modo migliore per mettere cariche frazionali sulle molecole, se vai a toccare mol.charges si incazza 
#  quando deve calcolare l' initial guess 
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [2]:
co=gto.Mole(atom="H 0 0 0 ;H 0 0 1.4", basis= "sto-3g",units="Bohr",maxit=100)
hfco=scf.RHF(co)
hfco.scf()
dL=.0001
hf1=fc(hfco,[dL,-dL])
hf1.scf()
hf2=fc(hfco,[-dL,dL])
hf2.scf()
dV=DeltaV(co,[dL,-dL])
print(np.allclose(dV,hf1.get_hcore()-hfco.get_hcore()))

Initialize <pyscf.gto.mole.Mole object at 0x7fbabc204150> in <pyscf.scf.hf.RHF object at 0x7fbabc241950>
<class 'pyscf.gto.mole.Mole'> does not have attributes  maxit units


converged SCF energy = -0.941480654707798
converged SCF energy = -0.941480680076734
converged SCF energy = -0.941480680076787
True


In [3]:
C=hfco.mo_coeff
S=hfco.get_ovlp()
e=hfco.mo_energy
E=np.diag(e)
O=np.diag(hfco.mo_occ)
C1=hf1.mo_coeff
P=hfco.make_rdm1()
P1=hf1.make_rdm1()
P2=hf2.make_rdm1() 
dP=(P1-P2)/2
dC=(abs(C1)*C/abs(C)-C)

In [4]:
h2=hfco.get_veff()
dh2=hf1.get_veff()-h2
J,K=hfco.get_jk()
g_ijkl=hfco.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
np.allclose(h2,np.einsum('ijkl,kl->ij',g_h2,P))

True

In [5]:
(C.T@S)@C, C@(C.T@S), (S@C)@C.T  # The inverse of C is C*S , and the inverse of C* is SC

(array([[1.00000000e+00, 2.22044605e-16],
        [1.11022302e-16, 1.00000000e+00]]),
 array([[1., 0.],
        [0., 1.]]),
 array([[1., 0.],
        [0., 1.]]))

In [6]:
U_app= C.T.dot(dV.dot(C))  #+U np.diag(np.diagonal(S)**-0.5) approximate value of 
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            U_app[i][j]/=(e[j]-e[i])
dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))
print(np.linalg.norm(dP_app-dP))

0.00035878765618787704
0.00041455727621003555
5.5769620022158494e-05


In [13]:
for i in range(10):
    U_app=C.T.dot((dV+np.einsum('ijkl,kl->ij',g_h2,dP_app)).dot(C)) # approximate value of U
    for i in range(co.nao):
        for j in range(co.nao):
            if e[i]!=e[j]:
                U_app[i][j]/=(e[j]-e[i])
    dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)
U=U_app.copy()
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))
print(np.linalg.norm(dP_app-dP))

0.00035878765618787704
0.00035878766055962415
4.3717471226277855e-12


In [56]:
ERI_mo=g_ijkl.copy()
ERI_mo=ERI_mo@C
ERI_mo=(ERI_mo.swapaxes(2,3)@C).swapaxes(3,2)
ERI_mo=(ERI_mo.swapaxes(1,3)@C).swapaxes(3,1)
ERI_mo=(ERI_mo.swapaxes(0,3)@C).swapaxes(3,0)

In [8]:
G_mo=g_h2.copy()
G_mo=G_mo@C
G_mo=(G_mo.swapaxes(2,3)@C).swapaxes(3,2)
G_mo=(G_mo.swapaxes(1,3)@C).swapaxes(3,1)
G_mo=(G_mo.swapaxes(0,3)@C).swapaxes(3,0)

In [57]:
G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1) -G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)

In [37]:
print(C.T@h2@C)
print((np.einsum('ijkl,kl->ij',G_mo,O)))
print(np.allclose(S@C@(np.einsum('ijkl,kl->ij',G_mo,O)@C.T@S),h2))

[[5.64818726e-01 2.77555756e-16]
 [2.77555756e-16 9.17322079e-01]]
[[5.64818726e-01 1.11022302e-16]
 [1.66533454e-16 9.17322079e-01]]
True


In [73]:
print(np.allclose(G_mo,ERI_mo-ERI_mo.swapaxes(1,2)/2))


True


In [86]:
print(np.einsum('ijkl,kl->ij',2*ERI_mo-ERI_mo.swapaxes(3,1)*0.5 -ERI_mo.swapaxes(1,2)*0.5,U@O))
print(np.einsum('ijkl,kl->ij',G_cphf,U@O)/2)

print(C.T@dh2@C)

#print((S@C@(np.einsum('ijkl,kl->ij',G_cphf,O))@C.T@S,dh2))

[[-2.49355400e-20 -1.19837973e-05]
 [-1.19837973e-05 -5.90084236e-20]]
[[-4.34340453e-20 -1.19837973e-05]
 [-1.19837973e-05 -6.06901059e-20]]
[[ 5.17619539e-09 -1.19837972e-05]
 [-1.19837972e-05 -4.72350141e-09]]


In [15]:
print(h2,C@(np.einsum('ijkl,kl->ij',G_mo,O)@C.T))

[[0.68895295 0.04288216]
 [0.04288216 0.68895295]] [[ 0.86918786 -0.43326972]
 [-0.43326972  0.86918786]]


In [16]:
print(np.einsum('ijkl,kl->ij',g_h2,dP))

[[ 1.14478954e-05 -2.87991202e-18]
 [-2.90024081e-18 -1.14478954e-05]]


In [17]:
print(np.einsum('ijkl,kl->ij',G_cphf.swapaxes(0,1),C@U@C.T))

[[ 1.36776851e-05  1.85517783e-05]
 [-1.36776851e-05  1.85517783e-05]]


In [18]:
print((np.einsum('ijkl,kl->ij',G_mo,U@C.T)))

[[-6.33604275e-05 -1.47308930e-05]
 [ 1.60573055e-05 -6.92486954e-05]]


In [19]:
C.T@dh2@C

array([[ 5.17619539e-09, -1.19837972e-05],
       [-1.19837972e-05, -4.72350141e-09]])

In [82]:
print(U)
print(U@O),print(O@U)
print(U@O-O@U)

[[-1.01643954e-20  1.21177980e-04]
 [-1.21177980e-04 -6.77626358e-21]]
[[-2.03287907e-20  0.00000000e+00]
 [-2.42355960e-04  0.00000000e+00]]
[[-2.03287907e-20  2.42355960e-04]
 [ 0.00000000e+00  0.00000000e+00]]
[[ 0.         -0.00024236]
 [-0.00024236  0.        ]]


In [81]:
print(np.einsum('ijkl,kl->ij',ERI_mo,U@O)*2)
print(np.einsum('ijkl,kl->ij',ERI_mo,U@O-O@U))

[[-6.33245384e-20 -1.08101465e-04]
 [-1.08101465e-04 -1.03902568e-19]]
[[-5.04504688e-20 -1.08101465e-04]
 [-1.08101465e-04 -6.72672918e-20]]


In [20]:
def fvind(x):
    x = x.reshape(Xvo.shape)
    if eris is None:
        mo_coeff = mycc.mo_coeff
        dm = reduce(numpy.dot, (mo_coeff[:,nocc:], x, mo_coeff[:,:nocc].T))
        dm = (dm + dm.T) * 2
        v = reduce(np.dot, (mo_coeff[:,nocc:].T, mycc._scf.get_veff(mol, dm), mo_coeff[:,:nocc]))
    else:
        v = numpy.zeros((nocc,nvir))
        for p0, p1 in prange(0, nocc, blksize):
            eris_ovov = _cp(eris.ovov[p0:p1])
            v[p0:p1] += numpy.einsum('iajb,bj->ia', eris_ovov, x) * 4
            v[p0:p1] -= numpy.einsum('ibja,bj->ia', eris_ovov, x)
            eris_ovov = None
            v[p0:p1] -= numpy.einsum('ijba,bj->ia', _cp(eris.oovv[p0:p1]), x)
    return v.T

In [21]:
def fvind(x):
    v = np.zeros((2,2))
    v += np.einsum('iajb,bj->ia', g_h2, x) * 4
    v -= np.einsum('ibja,bj->ia', g_h2, x)
    v -= np.einsum('ijba,bj->ia', g_h2, x)
    return v
    #return (4*np.einsum('ijkl,kl->ij',G_mo,P)-np.einsum('ijkl,jl->ik',G_mo,P))
dP2=cphf.solve(fvind,hfco.mo_energy,hfco.mo_occ,dV)
dP2[0]

array([[ 0.0009162,  0.       ],
       [ 0.       , -0.0009162]])

In [22]:
dP

array([[ 2.53701185e-04,  1.11022302e-16],
       [ 1.11022302e-16, -2.53701185e-04]])

In [23]:
dP_app

array([[ 2.53701188e-04, -8.13151629e-20],
       [ 1.08420217e-19, -2.53701188e-04]])

In [24]:
B=np.zeros((2,2))
for p in range(2):
    for q in range(2):
        B[p,q]=2*G_mo[p,q,0,0]-G_mo[p,0,q,0]
B

array([[2.82409363e-01, 3.81639165e-17],
       [8.32667268e-17, 9.79386032e-01]])

In [25]:
C.T.dot((np.einsum('ijkl,kl->ij',g_h2,dP_app)).dot(C))

array([[-3.38813179e-20, -1.19837973e-05],
       [-1.19837973e-05, -3.72694497e-20]])

In [26]:
np.einsum('ijkl,jl->ik',G_mo,U_app*dL)+np.einsum('ijkl,kl->ij',G_mo,U_app.T)*dL
B=np.zeros((2,2))
for q in range(2):
    for p in range(2):
        for i in range (2):
            for r in range(2):
                B[q,p]+=G_mo[q,r,p,i]*4*(U_app[r,i])
B

array([[-6.43647028e-21,  1.68267741e-04],
       [-1.68267741e-04, -4.59095349e-20]])

In [27]:
J

array([[1.13340783, 0.33026237],
       [0.33026237, 1.13340783]])

$$
H^{(2)}_{ij}=\sum_a^{n.occ.} \sum_{kl} g^{(2)}_{ijkl} C^\dagger_{ak} C_{la}$$

$$
 \sum_{ij} C^\dagger_{ai} \frac{\partial H^{(2)}_{ij}}{\partial Z}  C_{jb} =
 \sum_{ij} \left( \frac{\partial C^\dagger_{ai} }{\partial Z}  H^{(2)}_{ij}  C_{jb} +
  C^\dagger_{ai}  H^{(2)}_{ij} \frac{\partial C_{jb} }{\partial Z} \right)
$$
Che ha senso perchè gli integrali sono indipendenti da $Z$ e siccome l'operatore $V_{ee} $ è simmetrico in $r_1$ e $r_2$ posso applicare la derivata ai termini contratti all'interno, come ai termini contratti poi all'esterno !!! E quindi:

$$
 \sum_{ij} C^\dagger_{ai} \frac{\partial H^{(2)}_{ij}}{\partial Z}  C_{jb} =
 \sum_{k} \sum_{ij} \left(  U^\dagger_{ak} C^\dagger_{ki}    H^{(2)}_{ij}  C_{jb} +
  C^\dagger_{ai}  H^{(2)}_{ij}  C_{jk} U_{kb}  \right)
$$


In [28]:
j_c=g_h2.copy()
for i in range(2):
    for j in range(2):
          for k in range(2):
                for a in range(2):
                    j_c[i,j,k,a]=np.sum (j_c[i,j,k,:]*C[:,a]) 
for i in range(2):
    for j in range(2):
          for k in range(2):
                for a in range(2):
                    j_c[i,j,a,k]=np.sum (j_c[i,j,:,k]*C.T[a,:])                    
B=np.zeros((2,2))
for i in range(2):
    for j in range(2):
           B[i,j]=np.sum(j_c[i,j,:1,:1])       
#print(j_c )
print(B*2)
print(h2)

[[0.68895295 0.04288216]
 [0.04288216 0.68895295]]
[[0.68895295 0.04288216]
 [0.04288216 0.68895295]]


In [29]:
U=U_app
print(np.trace(np.einsum('ijkl,kl->ij',g_h2,P)@P))
print(np.trace(P@np.einsum('ijkl,kl->ij',g_h2,P))/2)
print(hfco.energy_elec())
print(C.T@h2@(C@U)+(U.T@C.T)@h2@C)
print()


#print(h2,np.einsum('ijkl,kl->ij',g_h2,P))
      #np.einsum('ijkl,kl->ij',G_mo,U))
#print(U_app.T@C.T@h2@C,C.T@h2@C@U_app )

1.1296374521092043
0.5648187260546021
(-1.3194643767935126, 0.564818726054602)
[[-6.77626358e-20 -4.27156443e-05]
 [-4.27156443e-05  6.77626358e-20]]



In [30]:
print(dir(hfco))

['CASCI', 'CASSCF', 'CCSD', 'CISD', 'DFMP2', 'DIIS', 'Gradients', 'MP2', 'QMMM', 'TDA', 'TDHF', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_chkfile', '_eigh', '_eri', '_finalize', '_is_mem_enough', '_keys', '_t0', '_w0', 'analyze', 'apply', 'as_scanner', 'build', 'callback', 'canonicalize', 'check_convergence', 'check_sanity', 'chkfile', 'conv_check', 'conv_tol', 'conv_tol_grad', 'converged', 'convert_from_', 'damp', 'damp_factor', 'density_fit', 'diis', 'diis_file', 'diis_space', 'diis_space_rollback', 'diis_start_cycle', 'dip_moment', 'direct_scf', 'direct_scf_tol', 'dump_chk', 'dump_flags', 'dump_scf_summary', 'e_tot', 'eig', 'energy_elec', 'energy_nuc', 'energy_tot', 'from_

In [31]:
"""A=np.linspace(0,7,8)
A=A.reshape(2,2,2)
C1=np.zeros((2,2,2))
B=np.asarray([[0,0],[1,1]])
print(A,"\n")
print((A.swapaxes(1,2)@B).swapaxes(2,1))
for i in range(2):
    for j in range(2):
          for k in range(2):
                C1[i,j,k]=np.sum(A[i,:,k]*B[:,j])    
print(C1)
"""
pass

In [34]:
dP,dP2,dP_app

(array([[ 2.53701185e-04,  1.11022302e-16],
        [ 1.11022302e-16, -2.53701185e-04]]),
 (array([[ 0.0009162,  0.       ],
         [ 0.       , -0.0009162]]),
  None),
 array([[ 2.53701188e-04, -8.13151629e-20],
        [ 1.08420217e-19, -2.53701188e-04]]))

In [35]:
dP2=dP2[0]*2

In [36]:
dEnn=(49-48)/2.1
print("true target energy: ",fc(hfco,[1,-1]).scf())
print("CO energy = ",hfco.e_tot)
print("APDFT1 energy =", hfco.e_tot+dEnn+np.trace(P.dot(dV/dL)))
print("APDFT2 energy =", hfco.e_tot+dEnn+np.trace((P+0.5*dP/dL).dot(dV/dL)))
print("APDFT2 energy approx.=", hfco.e_tot+dEnn+np.trace((P+0.5*dP_app/dL).dot(dV/dL)))
print("APDFT2 energy approx.=", hfco.e_tot+dEnn+np.trace((P+0.5*dP2/dL).dot(dV/dL)))

converged SCF energy = -2.61666811317645
true target energy:  -2.6166681131764524
CO energy =  -0.9414806547077983
APDFT1 energy = -0.4652901785173218
APDFT2 energy = -2.6242027018583483
APDFT2 energy approx.= -2.6242027281642133
APDFT2 energy approx.= -16.05839685076696
